In [9]:
import cv2
import numpy as np 
import os

### 1) Data preparation

In [10]:
dataset_path = 'C:/Users/91797/Desktop/'
face_data = []
labels = []
class_id = 0
name = {}

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        
        name [class_id] = fx[:-4]
        
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)
        
        # Create labels for the class 
        target = class_id * np.ones((data_item.shape[0]))
        class_id += 1
        labels.append(target)
        
face_dataset = np.concatenate(face_data,axis = 0)
face_label = np.concatenate(labels,axis = 0)
print(face_dataset.shape)
print(face_label.shape)
        

(22, 30000)
(22,)


### 2) KNN code

In [16]:
def dist (X1,X2):
    return np.sqrt(sum((X1-X2)**2))

def Knn (X,Y,Query_point, k = 5):
    vals = []
    
    m = X.shape[0]
    for i in range (m):
        d = dist(Query_point,X[i])
        vals.append((d,Y[i]))
        
    vals = sorted(vals)
    vals = vals[ : k]
    
    vals = np.array(vals)
    new_vals = np.unique(vals[ : , 1],return_counts = True)
    
    index = new_vals[1].argmax()
    pred = new_vals[0][index]
    
    return pred

### 3) Testing and face detection

##### Init camera

In [17]:
cap = cv2.VideoCapture(0)

##### Face detection

In [19]:
face_cascade = cv2.CascadeClassifier("C:/Users/91797/haarcascade_frontalface_alt.xml")

while True:
    ret,frame = cap.read()
    
    if ret == False:
        continue
        
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h = face
        
        # Extract or crop the required face section
        offset = 10
        face_section = frame[y - offset : y + offset + h , x - offset : x + offset + w]
        face_section = cv2.resize(face_section,(100,100))
        
        # Prediction
        out = Knn(face_dataset,face_label,face_section.flatten())
        pred_name = name [int(out)]
        
        # Giving Name and boundary to the frame
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
    cv2.imshow("Faces",frame)
        
    key_pressed = cv2.waitKey(1) & 0XFF  
    if key_pressed == ord('q') :
        break   
    
cap.release()
cv2.destroyAllWindows()
    